<a href="https://colab.research.google.com/github/balakrishna15/NLP_Projects/blob/main/NamedEntityRecognization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Named Entity Recognition Practice

In [2]:
import pandas as pd
data  = pd.read_csv("ner_dataset.csv",encoding = 'unicode_escape')
data.head(5)

,Unnamed: 0,Sentence #,Word,POS,Tag
0,0,Sentence: 1,Thousands,NNS,O
1,1,NaN,of,IN,O
2,2,NaN,demonstrators,NNS,O
3,3,NaN,have,VBP,O
4,4,NaN,marched,VBN,O


In [3]:
from itertools import chain
def make_dict_map(data, tokentag):
  token_to_idx = {}
  idx_to_token = {}
  # checking for tokentag to filter

  if tokentag == 'token':
    voc = list(set(data['Word'].to_list()))
  else:
    voc = list(set(data['Tag'].to_list()))

  idx_to_token = {idx:tok for idx, tok in enumerate(voc)}
  token_to_idx = {tok:idx for idx, tok in enumerate(voc)}
  return token_to_idx, idx_to_token


In [4]:
# Fitering the token and tag using the make_dict_map function

token_to_idx, idx_to_token = make_dict_map(data, 'token')
tag_to_idx, idx_to_tag = make_dict_map(data, 'tag')


In [5]:
#mapping the data with token and tag

data['Word_idx'] = data['Word'].map(token_to_idx)
data['Tag_idx'] = data['Tag'].map(tag_to_idx)

#Filling the NaN values in the dataset
data_fillna = data.fillna(method = 'ffill', axis =0)

In [6]:
data_group  = data_fillna.groupby(['Sentence #'], as_index = False)['Word','POS','Tag','Word_idx','Tag_idx'].agg(lambda x: list(x))

<ipython-input-6-7c846f5907da>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group  = data_fillna.groupby(['Sentence #'], as_index = False)['Word','POS','Tag','Word_idx','Tag_idx'].agg(lambda x: list(x))


In [7]:
data_group.head(5)

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[18055, 5103, 13617, 12274, 25636, 19387, 279,...","[1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 8, 1, 1, ..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[116, 16184, 15042, 238, 12668, 11838, 12006, ...","[15, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[10914, 18059, 8172, 6254, 6216, 16326, 12954,...","[1, 1, 12, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 7,..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[8509, 26261, 22910, 19976, 9953, 7186, 12051,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[19491, 5703, 4508, 4653, 3868, 7556, 10061, 1...","[8, 1, 1, 16, 3, 1, 12, 1, 8, 1, 15, 1, 15, 1,..."


In [8]:
from sklearn.model_selection import train_test_split

from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [11]:
def get_train_test_val(data_group, datas):

    #Creating pad_tokens (X var)
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    #getting the maximum token length and tag length
    ntoken = len(list(set(datas['Word'].to_list())))
    ntag = len(list(set(datas['Tag'].to_list())))

    padtokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= ntoken - 1)
    #Creating Pad Tags (y var) and converting into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    padtags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag_to_idx["O"])
    ntags = len(tag_to_idx)
    padtags = [to_categorical(i, num_classes=ntags) for i in padtags]

    #Splitting the train, test and validation set
    tokens, testtokens, tags, testtags = train_test_split(padtokens, padtags, test_size=0.1, train_size=0.9, random_state=2020)
    traintokens, valtokens, traintags, valtags = train_test_split(tokens,tags,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'length of train tokens :', len(traintokens),
        '\nlength of train tags   :', len(traintags),
        '\nlength of test tokens  :', len(testtokens),
        '\nlength of test tags    :', len(testtags),
        '\nlength of val tokens   :', len(valtokens),
        '\nlength of val tags     :', len(valtags),
    )

    return traintokens, testtokens, valtokens, traintags,testtags,valtags


In [12]:
#printing the lengths of train_tokens, test_tokens, val_tokens, train_tags,test_tags,val_tags
traintokens, testtokens, valtokens, traintags,testtags,valtags= get_train_test_val(data_group, data)

length of train tokens : 17543 
length of train tags   : 17543 
length of test tokens  : 2599 
length of test tags    : 2599 
length of val tokens   : 5848 
length of val tags     : 5848


In [13]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [14]:
#Finding the input and output dimension for Data
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])


In [16]:
ntags = len(tag_to_idx)
print(ntags)

18


In [17]:
#Function for the architecture of model.
def get_bilstmlstm():
    #selecting Sequential model
    model = Sequential()

    # Adding Embedding layer to the model
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Adding bidirectional LSTM to the model
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Adding LSTM to the model
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Adding timeDistributed Layer to the model
    model.add(TimeDistributed(Dense(ntags, activation="relu")))

    #Adding Adam optimizer to the model
    # Compile model
    #Adding Adam optimizer to the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [18]:
#Function to find the loss of the model
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [ ]:
#The Final output will be obtained after the 25 epochs as we set the loop to run 25 times
results = pd.DataFrame()
model_bilstm_lstm = get_bilstmlstm()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(traintokens, np.array(traintags), model_bilstm_lstm)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 104, 64)           1683136   
                                                                 
 bidirectional_1 (Bidirecti  (None, 104, 128)          66048     
 onal)                                                           
                                                                 
 lstm_3 (LSTM)               (None, 104, 64)           49408     
                                                                 
 time_distributed_1 (TimeDi  (None, 104, 18)           1170      
 stributed)                                                      
                                                                 
Total params: 1799762 (6.87 MB)
Trainable params: 1799762 (6.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
 8/15 [=========